In [13]:
%pip install prefect opentrons

  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
INFO: pip is looking at multiple versions of opentrons to determine which version is compatible with other requirements. This could take a while.
  Using cached opentrons-8.3.2-py2.py3-none-any.whl.metadata (5.1 kB)
INFO: pip is looking at multiple versions of opentrons to determine which version is compatible with other requirements. This could take a while.
  Using cached opentrons-8.3.2-py2.py3-none-any.whl.metadata (5.1 kB)
  Using cached opentrons_shared_data-8.3.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached opentrons-8.3.1-py2.py3-none-any.whl.metadata (5.1 kB)
  Using cached opentrons_shared_data-8.3.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached opentrons-8.3.0-py2.py3-none-any.whl.metadata (5.1 kB)
  Us

In [14]:
# source: https://support.opentrons.com/s/article/Running-the-robot-using-Jupyter-Notebook

# import opentrons.execute
import opentrons.simulate  # if you want to simulate the protocol first

# from opentrons.config import advanced_settings

# print(
#     "useProtocolEngine =",
#     advanced_settings.get_adv_setting("useProtocolEngine", "OT-2"),
# )


# This is where you establish the API version for executing a protocol
# protocol = opentrons.execute.get_protocol_api('2.11')
# to simulate the protocol, replace opentrons.execute with opentrons.simulate
# like so
protocol = opentrons.simulate.get_protocol_api("2.16")

# Must home before running
protocol.home()

# Labware
plate = protocol.load_labware("nest_96_wellplate_200ul_flat", "2")

tiprack_1 = protocol.load_labware("opentrons_96_tiprack_1000ul", location="1")

# Pipette
p1000 = protocol.load_instrument("p1000_single_gen2", "right", tip_racks=[tiprack_1])

# Protocol steps
p1000.transfer(100, plate["A1"], plate["A2"])

# Output steps in protocol
for line in protocol.commands():
    print(line)

22:23:13.376 | WARNING | opentrons.config.robot_configs - /Users/pod_0x000d/.opentrons/robot_settings.json not found. Loading defaults

22:23:13.398 | WARNING | opentrons.calibration_storage.ot2.deck_attitude - Deck calibration not found.

22:23:13.400 | WARNING | opentrons.config.robot_configs - /Users/pod_0x000d/.opentrons/deck_calibration.json not found. Loading defaults

Transferring 100.0 from A1 of NEST 96 Well Plate 200 µL Flat on slot 2 to A2 of NEST 96 Well Plate 200 µL Flat on slot 2
Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1
Aspirating 100.0 uL from A1 of NEST 96 Well Plate 200 µL Flat on slot 2 at 274.7 uL/sec
Dispensing 100.0 uL into A2 of NEST 96 Well Plate 200 µL Flat on slot 2 at 274.7 uL/sec
Dropping tip into Trash Bin on slot 12


In [ ]:
import json

import opentrons.simulate
from prefect import flow, task

protocol = plate = reservoir = tiprack_1 = tiprack_2 = p300 = None


@task
def init():
    global protocol, plate, reservoir, tiprack_1, tiprack_2, p300
    protocol = opentrons.simulate.get_protocol_api("2.16")

    protocol.home()  # home the robot

    # Define labware and pipettes
    # Load wireless charging port
    with open(
        # "/var/lib/jupyter/notebooks/ac_color_sensor_charging_port.json"
        "ac_color_sensor_charging_port.json"
    ) as labware_file1:
        labware_def1 = json.load(labware_file1)
        tiprack_2 = protocol.load_labware_from_definition(labware_def1, 10)

    # Load 3x2 vials rack
    with open(
        # "/var/lib/jupyter/notebooks/ac_6_tuberack_15000ul.json"
        "ac_6_tuberack_15000ul.json"
    ) as labware_file2:
        labware_def2 = json.load(labware_file2)
        reservoir = protocol.load_labware_from_definition(labware_def2, 3)

    # Load other labwares from Opentrons's labware library
    plate = protocol.load_labware(
        load_name="corning_96_wellplate_360ul_flat", location=1
    )
    tiprack_1 = protocol.load_labware(
        load_name="opentrons_96_tiprack_300ul", location=9
    )

    p300 = protocol.load_instrument(
        instrument_name="p300_single_gen2", mount="right", tip_racks=[tiprack_1]
    )

    p300.well_bottom_clearance.dispense = 8

    print("Labwares loaded")


@task
def mix_color(R, Y, B, mix_well):
    """Mix colors with specified RGB values into a well"""
    total = R + Y + B
    if total > 300:
        raise ValueError("The sum of the proportions must be not greater than 300")

    position = [
        "B1",
        "B2",
        "B3",
    ]  # R,Y,B paint vial on the position of the 3x2 vials rack
    portion = {"B1": R, "B2": Y, "B3": B}

    red_volume = R
    yellow_volume = Y
    blue_volume = B
    color_volume = {"B1": red_volume, "B2": yellow_volume, "B3": blue_volume}

    # assert all(i is not None for i in [p300, tiprack_1, reservoir, plate])
    # doing this for pylance
    assert p300 is not None
    assert tiprack_1 is not None
    assert reservoir is not None
    assert plate is not None

    for pos in position:
        if float(portion[pos]) != 0.0:  # if zero, skip this color aspiration process
            p300.pick_up_tip(tiprack_1[pos])
            p300.aspirate(color_volume[pos], reservoir[pos])
            p300.dispense(color_volume[pos], plate[mix_well])
            p300.default_speed = 100
            # reduce pipette speed to prevent droplets falling, speed unit: mm/sec
            p300.blow_out(reservoir["A1"].top(z=-5))
            p300.default_speed = (
                400  # reset pipette speed, speed unit: mm/sec, 400 is default value
            )
            p300.drop_tip(tiprack_1[pos])

    print(f"Mixed R:{R}, Y:{Y}, B:{B} in well {mix_well}")


@task
def move_sensor_to_measurement_position(mix_well):
    """Move sensor to measurement position"""
    # Move sensor to measurement position
    assert p300 is not None
    assert tiprack_2 is not None
    assert plate is not None
    # p300.pick_up_tip(tiprack_2["A2"])
    p300.move_to(tiprack_2["A2"].top())
    p300.move_to(plate[mix_well].top(z=-1.3))  # z=-1.3 to get sensor closer to the well
    print("Sensor is now in position for measurement")


@task
def move_sensor_back():
    """Move sensor back to charging position"""
    assert p300 is not None
    assert tiprack_2 is not None
    # p300.drop_tip(tiprack_2["A2"].top(z=-80))
    p300.move_to(tiprack_2["A2"].top(z=-80))
    print("Sensor moved back to charging position")


# `SequentialTaskRunner` is used to run tasks sequentially (sync).
@flow(name="main_flow")
def main_flow(R: int = 100, Y: int = 100, B: int = 100, mix_well: str = "A1"):
    """
    Main flow is basically a wrapper for `mix_color` task.
    This is because `mix_color` is now a `task` and can be used in a `flow`.
    """
    init()
    mix_color(R, Y, B, mix_well)
    move_sensor_to_measurement_position(mix_well)
    move_sensor_back()


if __name__ == "__main__":
    ## Below cannot happen in Jupyter Notebook, but can be used in Python script.
    main_flow.serve(
        name="my-first-deployment",
        tags=["onboarding"],
        parameters={"goodbye": True},
        interval=60,
    )

    ## That's why we use `main_flow()` directly here.
    main_flow()

Your flow 'main_flow' is being served and polling for scheduled runs!

To trigger a run for this flow, use the following command:

        $ prefect deployment run 'main_flow/my-first-deployment'

You can also run your flow via the Prefect UI: https://app.prefect.cloud/account/5b838504-64cf-4297-9b35-b881ac6169b3/workspace/d2718b4c-b49a-43ce-83c2-baf6fb3b9665/deployments/deployment/<coroutine object sync_compatible.<locals>.coroutine_wrapper.<locals>.ctx_call at 0x313971b40>

RuntimeError: This event loop is already running

## Code Backup

In [ ]:
# # from prefect.runner.storage import GitRepository, GitCredentials

# from prefect import serve

# from demo import demo_flow
# from final_state_from_task import running_flow

# from ot2mqtt import mix_color, move_sensor_to_measurement_position, move_sensor_back

# serve(
#     mix_color.to_deployment("mix-color"),  # generates OpenAPI schema
#     move_sensor_to_measurement_position.to_deployment(
#         "move-sensor-to-measurement-position"
#     ),
#     move_sensor_back.to_deployment("move-sensor-back"),
# )

ModuleNotFoundError: No module named 'demo'

In [ ]:
# test= running_flow.from_source(
#         source=GitRepository(
#             url="https://github.com/masonmenges/mm2-sanbox.git"
#             ),
#         entrypoint="flows/final_state_from_task.py:runni
# )

https://docs.prefect.io/v3/deploy/deployment-versioning#git-hub-actions